<h3> Importation des librairies et sélection des options d'affichage </h3>

In [1]:
import networkx
import gurobipy
import matplotlib.pyplot as plt
import sys
import random
import csv

options = {
    "font_size": 20,
    "node_size": 1000,
    "node_color": "white",
    "edgecolors": "black",
    "linewidths": 5,
    "width": 5,
    "with_labels": True,
}

In [ ]:
# Generer des instances de graphes

def generate_graph(graph_type, n_nodes, p=None):
    """
    Génère un graphe selon le type spécifié.
    :param graph_type: Type de graphe ('complete', 'cycle', 'random').
    :param n_nodes: Nombre de nœuds.
    :param p: Probabilité pour les graphes aléatoires (nécessaire pour 'random').
    :return: Graphe généré.
    """
    if graph_type == 'complete':
        G = nx.complete_graph(n_nodes, create_using=nx.DiGraph)
    elif graph_type == 'cycle':
        G = nx.cycle_graph(n_nodes, create_using=nx.DiGraph)
    elif graph_type == 'random':
        G = nx.erdos_renyi_graph(n_nodes, p, directed=True)
    else:
        raise ValueError("Type de graphe non supporté")
    return G


def assign_capacities_and_costs(G, capacity_range, cost_range):
    """
    Assigne des capacités et des coûts de routage aux arcs.
    :param G: Graphe.
    :param capacity_range: Intervalle pour les capacités (e.g., (1, 10)).
    :param cost_range: Intervalle pour les coûts (e.g., (1, 5)).
    :return: Graphe avec attributs assignés.
    """
    for u, v in G.edges:
        G[u][v]['capacity'] = random.randint(*capacity_range)
        G[u][v]['cost'] = random.uniform(*cost_range)
    return G


def generate_traffic_demand(G, S_size, demand_range):
    """
    Génère des sous-ensembles S de nœuds et des demandes de trafic.
    :param G: Graphe.
    :param S_size: Taille de l'ensemble S.
    :param demand_range: Intervalle pour les demandes de trafic (e.g., (1, 20)).
    :return: Ensemble S et vecteur des demandes de trafic.
    """
    nodes = list(G.nodes)
    S = random.sample(nodes, S_size)
    traffic_demand = {}
    for i in S:
        for j in S:
            if i != j:
                traffic_demand[(i, j)] = random.randint(*demand_range)
    return S, traffic_demand


def save_to_csv(G, S, traffic_demand, filename):
    """
    Sauvegarde le graphe, les capacités, les coûts et les demandes de trafic dans un fichier CSV.
    :param G: Graphe.
    :param S: Sous-ensemble des nœuds.
    :param traffic_demand: Dictionnaire des demandes de trafic.
    :param filename: Nom du fichier CSV.
    """
    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        # Écriture des arcs avec capacités et coûts
        writer.writerow(['Source', 'Target', 'Capacity', 'Cost'])
        for u, v in G.edges:
            writer.writerow([u, v, G[u][v]['capacity'], G[u][v]['cost']])
        
        # Écriture des demandes de trafic
        writer.writerow([])
        writer.writerow(['Source', 'Target', 'Traffic Demand'])
        for (i, j), demand in traffic_demand.items():
            writer.writerow([i, j, demand])
        
        # Écriture de l'ensemble S
        writer.writerow([])
        writer.writerow(['Subset S'])
        writer.writerow(S)


# Exemple d'utilisation
if __name__ == "__main__":
    # Paramètres
    graph_type = 'random'  # Type de graphe : 'complete', 'cycle', 'random'
    n_nodes = 10  # Nombre de nœuds
    capacity_range = (1, 10)  # Capacités des arcs
    cost_range = (1.0, 5.0)  # Coûts de routage
    S_size = 5  # Taille de l'ensemble S
    demand_range = (1, 20)  # Intervalle des demandes de trafic
    filename = 'graph_instance.csv'  # Nom du fichier CSV

    # Génération du graphe
    G = generate_graph(graph_type, n_nodes, p=0.3)

    # Assignation des capacités et des coûts
    G = assign_capacities_and_costs(G, capacity_range, cost_range)

    # Génération de l'ensemble S et des demandes
    S, traffic_demand = generate_traffic_demand(G, S_size, demand_range)

    # Sauvegarde dans un fichier CSV
    save_to_csv(G, S, traffic_demand, filename)

    print(f"Instance générée et sauvegardée dans {filename}")